In [ ]:
import random

# **Podpunkt 1.a**

In [ ]:
def edit_distance(x: str, y: str, costs: list):

    insert_cost, delete_cost, replace_cost, swap_cost = costs  # Przypisujemy koszty z wektora

    m, n = len(x), len(y)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Inicjalizacja pierwszego wiersza i pierwszej kolumny
    for i in range(m + 1):
        dp[i][0] = i * delete_cost
    for j in range(n + 1):
        dp[0][j] = j * insert_cost

    # Wypełnianie macierzy DP
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if x[i - 1] == y[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # Bez kosztu, jeśli znaki są takie same
            else:
                dp[i][j] = min(
                    dp[i - 1][j] + delete_cost,  # Usunięcie znaku
                    dp[i][j - 1] + insert_cost,  # Wstawienie znaku
                    dp[i - 1][j - 1] + replace_cost  # Zamiana znaku
                )

    return dp[m][n]

# Przykład użycia
# Lista kosztów: [insert_cost, delete_cost, replace_cost, transposition_cost]
costs = [1, 1, 1, 1]

x = "kot"
y = "aforyzm"
cost = edit_distance(x, y, costs)
print(f"Minimalny koszt przekształcenia '{x}' w '{y}' z zamianą sąsiednich znaków: {cost}")


Minimalny koszt przekształcenia 'kot' w 'aforyzm' z zamianą sąsiednich znaków: 6


# **Podpunkt 1.b**

In [ ]:
def edit_distance_with_transposition(x: str, y: str, costs: list, verbose = True):

    insert_cost, delete_cost, replace_cost, swap_cost = costs  # Przypisujemy koszty z wektora

    m, n = len(x), len(y)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Inicjalizacja pierwszego wiersza i pierwszej kolumny
    for i in range(m + 1):
        dp[i][0] = i * delete_cost
    for j in range(n + 1):
        dp[0][j] = j * insert_cost

    # Wypełnianie macierzy DP
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if x[i - 1] == y[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # Bez kosztu, jeśli znaki są takie same
             # Sprawdzamy możliwość transpozycji
            elif i > 1 and j > 1 and x[i - 1] == y[j - 2] and x[i - 2] == y[j - 1]:
                dp[i][j] = min(dp[i][j], dp[i - 2][j - 2] + swap_cost)
            else:
                dp[i][j] = min(
                    dp[i - 1][j] + delete_cost,  # Usunięcie znaku
                    dp[i][j - 1] + insert_cost,  # Wstawienie znaku
                    dp[i - 1][j - 1] + replace_cost  # Zamiana znaku
                )

    if verbose:
        print("Macierz kosztów transformacji:")
        for row in dp:
            print(row)

    return dp[m][n]

# Lista kosztów: [insert_cost, delete_cost, replace_cost, transposition_cost]
costs = [1, 1, 2, 1]

x = "babcia"
y = "album"
cost = edit_distance_with_transposition(x, y, costs)
print(f"Minimalny koszt przekształcenia '{x}' w '{y}' z zamianą sąsiednich znaków: {cost}")


Macierz kosztów transformacji:
[0, 1, 2, 3, 4, 5]
[1, 2, 3, 2, 3, 4]
[2, 1, 2, 3, 4, 5]
[3, 2, 3, 2, 3, 4]
[4, 3, 4, 3, 4, 5]
[5, 4, 5, 4, 5, 6]
[6, 5, 6, 5, 6, 7]
Minimalny koszt przekształcenia 'babcia' w 'album' z zamianą sąsiednich znaków: 7


# **Podpunkt 1 Lista3**

In [ ]:
import numpy as np

# Układ klawiatury QWERTY
keyboard_layout = {
    'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3), 't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
    'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3), 'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8),
    'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3), 'b': (2, 4), 'n': (2, 5), 'm': (2, 6)
}

def key_distance(char1, char2):
    """Oblicza euklidesową odległość między dwoma klawiszami na klawiaturze."""
    pos1, pos2 = keyboard_layout.get(char1), keyboard_layout.get(char2)
    if pos1 and pos2:
        return np.linalg.norm(np.array(pos1) - np.array(pos2)) * (2 / 9)
    return 2  # Maksymalny koszt zamiany

def average_neighbor_distance(char):
    """Oblicza średnią odległość do sąsiednich klawiszy."""
    if char not in keyboard_layout:
        return 1  # Domyślny koszt dla nieznanych znaków
    x, y = keyboard_layout[char]
    neighbors = []

    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        neighbor = (x + dx, y + dy)
        for key, pos in keyboard_layout.items():
            if pos == neighbor:
                neighbors.append(key_distance(char, key))

    return np.mean(neighbors) if neighbors else 1  # Średnia odległość sąsiadów

def min_edit_distance_qwerty(x, y):
    """Oblicza minimalny koszt przekształcenia x w y zgodnie z kosztami klawiatury QWERTY."""
    m, n = len(x), len(y)

    # Tworzymy macierz dynamiczną
    dp = np.zeros((m+1, n+1))

    # Inicjalizacja kosztów operacji edycyjnych
    for i in range(1, m+1):
        dp[i][0] = dp[i-1][0] + average_neighbor_distance(x[i-1])  # Koszt usunięcia znaku

    for j in range(1, n+1):
        dp[0][j] = dp[0][j-1] + 1  # Koszt dodania znaku (stały)

    # Wypełnianie macierzy dynamicznie
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(
                    dp[i-1][j] + average_neighbor_distance(x[i-1]),  # Usunięcie
                    dp[i][j-1] + 1,  # Wstawienie
                    dp[i-1][j-1] + key_distance(x[i-1], y[j-1])  # Zamiana
                )

    # Normalizacja macierzy
    mean_weight = np.mean(dp)
    dp /= mean_weight

    return dp[m][n], dp  # Minimalny koszt i macierz kosztów

# Przykład użycia:
x = "kot"
y = "pies"

cost, dp_matrix = min_edit_distance_qwerty(x, y)
print(f"Minimalny koszt przekształcenia '{x}' w '{y}': {cost}")
print("Macierz dynamiczna kosztów:")
print(dp_matrix)


Minimalny koszt przekształcenia 'kot' w 'pies': 1.4903763460612278
Macierz dynamiczna kosztów:
[[0.         0.68885032 1.37770064 2.06655096 2.75540129]
 [0.15307785 0.34229248 0.84192817 1.53077849 2.21962881]
 [0.3061557  0.3061557  0.49537033 1.18422065 1.87307097]
 [0.45923355 0.45923355 0.64844818 0.80152602 1.49037635]]


# **Podpunkt 2 Lista3**

In [ ]:
import numpy as np

# Układ klawiatury QWERTY
keyboard_layout = {
    'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3), 't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
    'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3), 'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8),
    'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3), 'b': (2, 4), 'n': (2, 5), 'm': (2, 6)
}

def are_neighbors(char1, char2):
    """Sprawdza, czy dwa klawisze są sąsiadami na klawiaturze."""
    x1, y1 = keyboard_layout.get(char1)
    x2, y2 = keyboard_layout.get(char2)

    # Sprawdzamy, czy są sąsiadami (różnica w pozycji x lub y wynosi 1)
    if x1 is not None and x2 is not None:
        return abs(x1 - x2) <= 1 and abs(y1 - y2) <= 1
    return False

def min_edit_distance_with_weights(x, y, x_weight, y_weight):
    """Oblicza minimalny koszt przekształcenia x w y z wagami dla sąsiadów (x) i nienaosnych (y)."""
    m, n = len(x), len(y)

    # Tworzymy macierz dynamiczną
    dp = np.zeros((m+1, n+1))

    # Inicjalizacja kosztów operacji edycyjnych
    for i in range(1, m+1):
        dp[i][0] = dp[i-1][0] + average_neighbor_distance(x[i-1])  # Koszt usunięcia znaku

    for j in range(1, n+1):
        dp[0][j] = dp[0][j-1] + 1  # Koszt dodania znaku (stały)

    # Wypełnianie macierzy dynamicznie
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                # Wybieramy wagę x jeśli klawisze są sąsiadami, inaczej wagę y
                if are_neighbors(x[i-1], y[j-1]):
                    replace_cost = x_weight
                else:
                    replace_cost = y_weight

                dp[i][j] = min(
                    dp[i-1][j] + average_neighbor_distance(x[i-1]),  # Usunięcie
                    dp[i][j-1] + 1,  # Wstawienie
                    dp[i-1][j-1] + replace_cost  # Zamiana
                )

    # Normalizacja macierzy
    mean_weight = np.mean(dp)
    dp /= mean_weight

    return dp[m][n], dp  # Minimalny koszt i macierz kosztów

# Przykład użycia:
x = "kot"
y = "pies"
x_weight = 1  # Koszt dla sąsiadów
y_weight = 2  # Koszt dla nienaosnych klawiszy

cost, dp_matrix = min_edit_distance_with_weights(x, y, x_weight, y_weight)
print(f"Minimalny koszt przekształcenia '{x}' w '{y}': {cost}")
print("Macierz dynamiczna kosztów:")
print(dp_matrix)


Minimalny koszt przekształcenia 'kot' w 'pies': 2.0100502512562817
Macierz dynamiczna kosztów:
[[0.         0.45226131 0.90452261 1.35678392 1.80904523]
 [0.10050251 0.55276382 0.90452261 1.35678392 1.80904523]
 [0.20100503 0.55276382 1.00502513 1.45728643 1.90954774]
 [0.30150754 0.65326633 1.10552764 1.55778894 2.01005025]]


# **Podpunkt 2 Lista3** dodana transpozycja

In [ ]:
import numpy as np

# Układ klawiatury QWERTY
keyboard_layout = {
    'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3), 't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
    'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3), 'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8),
    'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3), 'b': (2, 4), 'n': (2, 5), 'm': (2, 6)
}

def are_neighbors(char1, char2):
    """Sprawdza, czy dwa klawisze są sąsiadami na klawiaturze."""
    x1, y1 = keyboard_layout.get(char1)
    x2, y2 = keyboard_layout.get(char2)

    # Sprawdzamy, czy są sąsiadami (różnica w pozycji x lub y wynosi 1)
    if x1 is not None and x2 is not None:
        return abs(x1 - x2) <= 1 and abs(y1 - y2) <= 1
    return False

def key_distance(char1, char2):
    """Oblicza euklidesową odległość między dwoma klawiszami na klawiaturze."""
    pos1, pos2 = keyboard_layout.get(char1), keyboard_layout.get(char2)
    if pos1 and pos2:
        return np.linalg.norm(np.array(pos1) - np.array(pos2)) * (2 / 9)
    return 2  # Maksymalny koszt zamiany

def min_edit_distance_with_transposition(x, y, x_weight, y_weight):
    """Oblicza minimalny koszt przekształcenia x w y z wagami dla sąsiadów (x) i nienaosnych (y) oraz transpozycją."""
    m, n = len(x), len(y)

    # Tworzymy macierz dynamiczną
    dp = np.zeros((m+1, n+1))

    # Inicjalizacja kosztów operacji edycyjnych
    for i in range(1, m+1):
        dp[i][0] = dp[i-1][0] + average_neighbor_distance(x[i-1])  # Koszt usunięcia znaku

    for j in range(1, n+1):
        dp[0][j] = dp[0][j-1] + 1  # Koszt dodania znaku (stały)

    # Wypełnianie macierzy dynamicznie
    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                # Wybieramy wagę x jeśli klawisze są sąsiadami, inaczej wagę y
                if are_neighbors(x[i-1], y[j-1]):
                    replace_cost = x_weight
                else:
                    replace_cost = y_weight

                # Uwzględniamy również możliwość transpozycji
                transposition_cost = np.inf
                if i > 1 and j > 1 and x[i-1] == y[j-2] and x[i-2] == y[j-1]:
                    transposition_cost = dp[i-2][j-2] + replace_cost

                dp[i][j] = min(
                    dp[i-1][j] + average_neighbor_distance(x[i-1]),  # Usunięcie
                    dp[i][j-1] + 1,  # Wstawienie
                    dp[i-1][j-1] + replace_cost,  # Zamiana
                    transposition_cost  # Transpozycja
                )

    # Normalizacja macierzy
    mean_weight = np.mean(dp)
    dp /= mean_weight

    return dp[m][n], dp  # Minimalny koszt i macierz kosztów

# Przykład użycia:
x = "algorytm"
y = "aforyzm"
x_weight = 1  # Koszt dla sąsiadów
y_weight = 2  # Koszt dla nienaosnych klawiszy

cost, dp_matrix = min_edit_distance_with_transposition(x, y, x_weight, y_weight)
print(f"Minimalny koszt przekształcenia '{x}' w '{y}': {cost}")
print("Macierz dynamiczna kosztów:")
print(dp_matrix)


Minimalny koszt przekształcenia 'algorytm' w 'aforyzm': 1.0044388078630313
Macierz dynamiczna kosztów:
[[0.         0.41090679 0.82181357 1.23272036 1.64362714 2.05453393
  2.46544071 2.8763475 ]
 [0.09131262 0.         0.41090679 0.82181357 1.23272036 1.64362714
  2.05453393 2.46544071]
 [0.18262524 0.09131262 0.5022194  0.82181357 1.23272036 1.64362714
  2.05453393 2.46544071]
 [0.27393786 0.18262524 0.5022194  0.91312619 1.23272036 1.64362714
  2.05453393 2.46544071]
 [0.36525048 0.27393786 0.59353202 0.5022194  0.91312619 1.32403297
  1.73493976 2.14584654]
 [0.45656309 0.36525048 0.68484464 0.59353202 0.5022194  0.91312619
  1.32403297 1.73493976]
 [0.54787571 0.45656309 0.77615726 0.68484464 0.59353202 0.5022194
  0.91312619 1.32403297]
 [0.63918833 0.54787571 0.86746988 0.77615726 0.68484464 0.59353202
  1.00443881 1.41534559]
 [0.73050095 0.63918833 0.9587825  0.86746988 0.77615726 0.68484464
  1.09575143 1.00443881]]


# **Podpunkt 3 Lista3**

In [ ]:
import numpy as np
from collections import Counter

# Przykładowa mapa odległości między literami na klawiaturze (można ją rozbudować)
keyboard_distance = {
    ('a', 's'): 1, ('s', 'd'): 1, ('d', 'f'): 1, ('f', 'g'): 1,
    ('g', 'h'): 1, ('h', 'j'): 1, ('j', 'k'): 1, ('k', 'l'): 1,
    ('a', 'q'): 1, ('s', 'w'): 1, ('d', 'e'): 1,  # itd.
    # Możesz dodać pełną mapę odległości na podstawie układu klawiatury
}

def normalize_frequency(word):
    # Zliczanie wystąpień liter
    freq = Counter(word)
    # Obliczanie średniej liczby wystąpień
    avg_freq = sum(freq.values()) / len(freq)
    # Zwracanie znormalizowanego kosztu (ω(A) = wystąpienie / średnia)
    return {char: count / avg_freq for char, count in freq.items()}

def edit_distance(x: str, y: str, costs: list):
    insert_cost, delete_cost, replace_cost, swap_cost = costs  # Koszty operacji

    # Zliczanie wystąpień w każdym słowie i normalizacja
    freq_x = normalize_frequency(x)
    freq_y = normalize_frequency(y)

    m, n = len(x), len(y)
    dp = np.zeros((m + 1, n + 1))

    # Inicjalizacja pierwszego wiersza i pierwszej kolumny
    for i in range(m + 1):
        dp[i][0] = i * delete_cost
    for j in range(n + 1):
        dp[0][j] = j * insert_cost

    # Wypełnianie macierzy DP
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            char_x = x[i - 1]
            char_y = y[j - 1]

            if char_x == char_y:
                dp[i][j] = dp[i - 1][j - 1]  # Bez kosztu, jeśli znaki są takie same
            else:
                # Koszt wstawienia
                insert = dp[i][j - 1] + insert_cost * freq_y.get(char_y, 1)
                # Koszt usunięcia
                delete = dp[i - 1][j] + delete_cost * freq_x.get(char_x, 1)
                # Koszt zamiany
                replace = dp[i - 1][j - 1] + replace_cost * (abs(ord(char_x) - ord(char_y)) / 100) + freq_y.get(char_y, 1)

                dp[i][j] = min(insert, delete, replace)

            # Możliwość uwzględnienia zamiany sąsiednich znaków (swap)
            if i > 1 and j > 1 and x[i - 1] == y[j - 2] and x[i - 2] == y[j - 1]:
                dp[i][j] = min(dp[i][j], dp[i - 2][j - 2] + swap_cost)

    return dp[m][n]

# Przykład użycia
costs = [1, 1, 1, 1]
x = "kot"
y = "pies"
cost = edit_distance(x, y, costs)
print(f"Minimalny koszt przekształcenia '{x}' w '{y}': {cost}")


Minimalny koszt przekształcenia 'kot' w 'pies': 4.12
